In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import os
import csv

# for progress bars
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())

# API Key
from config import api_key

#ZillowWrapper
from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults

0it [00:00, ?it/s]


In [3]:
#Read csv
general_df = pd.read_csv('test_zillow_info.csv')
general_df.head()

,Address,Zip,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,5 14th,60605,60204228.0,248760.0,41.8682,NaN,Condominium,1250.0,2.0,2.0,05/27/2004,295000.0,295000.0,393921.0,04/02/2019,49273.0
1,8 9th,60605,119088436.0,40600.0,41.8681,NaN,Miscellaneous,NaN,NaN,0.0,09/29/2017,205000.0,205000.0,NaN,01/01/1970,NaN
2,234 Ashland,60605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1733 Congress,60605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1753 Congress,60605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
general_df = pd.read_csv('test_zillow_info.csv').dropna(subset=['Zillow ID','Zestimate Amount'])
general_df.head()

In [ ]:
#Scatter Plot: Zestimate Value Change by Zip and Home Type

In [ ]:
#Zestimate Amount / HomeSize by Zip and Home Type 

In [ ]:
#Amount Change by Year by Zip and Home Type